<a href="https://colab.research.google.com/github/YasminBelem/Chat-bot-Lumina---Imersao-Alura-Google-Gemini/blob/main/Chat_bot_Lumina_Projeto_Imers%C3%A3o_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configuração da API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configuração do cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [53]:
# Instalação Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [63]:
##########################################
# --- Agente 1: Buscador de Artigos --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-pro-preview-03-25",

        instruction="""
        Você é um buscador especializado em artigos científicos acadêmicos. Sua tarefa é realizar uma busca abrangente utilizando a ferramenta 'google_search' em bases de dados oficiais e confiáveis,
        como SciELO (Scientific Electronic Library Online), LILACS (Literatura Latino-Americana e do Caribe em Ciências da Saúde), PubMed (para artigos da área da saúde), Web of Science, Scopus,
        e outras relevantes para pesquisa científica.
        O usuário fornecerá um tópico de pesquisa. Sua busca deve retornar pelo menos 8 artigos científicos que atendam aos seguintes critérios:
          1. Relevância Científica: Os artigos devem ser resultados de pesquisa científica original, revisões sistemáticas, ou meta-análises que abordem diretamente o tópico fornecido pelo usuário ou
          apresentem informações cientificamente relevantes para a compreensão do tema.
          2. Data de Publicação Recente: Os artigos devem ter sido publicados nos últimos 10 anos, considerando a data atual de busca. Artigos publicados antes dessa data não devem ser considerados,
          a menos que sejam reconhecidamente seminais e metodologicamente cruciais para o entendimento do tópico (nesse caso, justifique brevemente sua inclusão com foco em seu impacto científico).
          3. Fonte Científica Confiável: Priorize artigos publicados em periódicos científicos com rigoroso processo de revisão por pares e indexados nas bases de dados mencionadas ou em outras de
          reputação científica similar. Exclua resultados de fontes não acadêmicas ou de baixa credibilidade científica.
        Para cada um dos 8 artigos (ou mais, se encontrar relevantes), forneça o título completo do artigo científico, autor(es), data de publicação (se tiver), número de páginas, periódico ou revista em que foi publicado, volume (se tiver)
        e o link direto (URL) para acessar o artigo na íntegra.
        Certifique-se de sempre usar fontes confiáveis e relevantes.
        """,
        description="Agente que busca artigos científicos.",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    artigos = call_agent(buscador, entrada_do_agente_buscador)
    return artigos

In [55]:
################################################
# --- Agente 2: Redator de resumos --- #
################################################
def agente_redator_de_resumos(topico, artigos_buscados):
    redator = Agent(
        name="agente_redator_de_resumos",
        model="gemini-2.0-flash",

        instruction="""
        Você é um redator especializado em elaborar resumos concisos e informativos de artigos científicos acadêmicos. Com base na lista de artigos científicos encontrados pelo buscador,
        para cada artigo listado, sua tarefa é produzir um resumo que capture os pontos essenciais do trabalho científico. O resumo deve seguir as seguintes diretrizes, sem reescrever o
        artigo extensivamente: 1. Identificação Clara: Comece o resumo indicando o título do artigo científico. 2. Objetivo da Pesquisa Científica: Mencione qual era o objetivo principal da pesquisa
        científica apresentada no artigo. 3. Metodologia Científica (se aplicável): Descreva brevemente a metodologia científica utilizada pelos autores para conduzir a pesquisa (tipo de estudo,
        participantes, coleta de dados, análise estatística ou qualitativa, etc.), apenas se for crucial para entender os resultados científicos. 4. Resultados Científicos Centrais: Apresente os
        principais resultados ou achados da pesquisa científica de forma clara e concisa. Mencione dados quantitativos ou qualitativos chave, se relevantes para a compreensão científica.
        5. Conclusões Científicas: Resuma as principais conclusões dos autores e suas implicações científicas, sem interpretá-las excessivamente. 6. Linguagem Clara e Concisa: Utilize uma linguagem
        acessível a estudantes universitários, evitando jargões excessivos e explicações longas. O objetivo é fornecer uma visão geral rápida e compreensível do artigo científico, mantendo a integridade
        das informações científicas originais. Mantenha cada resumo dentro de um limite razoável de palavras (por exemplo, de no mínimo 100 e máximo de 200 palavras por artigo).
        Tente focar nas informações relevantes que o artigo traz para o tópico que o usuário digitou.
        """,
        description="Agente que escreve resumos de artigos cientifícos."
    )

    entrada_do_agente_redator_de_resumos = f"Tópico:{topico}\nArtigos encontrados: {artigos_buscados}"
    resumo = call_agent(redator, entrada_do_agente_redator_de_resumos)
    return resumo

In [56]:
#############################################
# --- Agente 3: Captador de citações --- #
#############################################
def agente_captador_de_citacoes(topico, artigos_buscados):
    captador = Agent(
        name="agente_captador_de_citacoes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em identificar e formatar citações diretas relevantes de artigos científicos acadêmicos buscados, seguindo as normas da ABNT (Associação Brasileira de Normas Técnicas).
        Analise cuidadosamente o conteúdo dos artigos científicos encontrados anteriormente e, para cada artigo, sua tarefa é selecionar pelo menos 5 citações diretas que atendam aos seguintes
        critérios: 1. Relevância Científica para o Tópico: As citações devem abordar aspectos importantes, conceitos chave científicos ou resultados científicos significativos relacionados ao
        tópico de pesquisa do usuário. 2. Potencial de Uso Acadêmico Científico: Escolha citações que apresentem informações científicas concisas e que possam ser utilizadas diretamente para embasar
        diferentes partes de um trabalho acadêmico científico (introdução, revisão de literatura, discussão, etc.). 3. Interesse Científico: Selecione citações que apresentem insights científicos
        relevantes, dados científicos impactantes ou afirmações científicas que possam gerar discussão científica. As citações podem conter ser pequenas ou de tamanho intermediário (preferêncialmente), porém nunca grandes demais.
        Formate cada citação direta rigorosamente de acordo com as normas da ABNT para citações, localizando no artigo a página das citações.
        """,
        description="Agente captador de citações para utilizar em textos acadêmicos."
    )
    entrada_do_agente_captador_de_citacoes = f"Tópico: {topico}\nArtigos encontrados: {artigos_buscados}"
    citacoes = call_agent(captador, entrada_do_agente_captador_de_citacoes)
    return citacoes

In [57]:
###########################################################
# --- Agente 4: Formatador de Referências em ABNT --- #
###########################################################
def agente_formatador_de_referencias(artigos_buscados, data_de_hoje):
    formatador = Agent(
        name="agente_formatador_de_referencias",
        model="gemini-2.0-flash",
        instruction="""
        Você é um bibliotecário especializado em formatação de referências bibliográficas de artigos científicos encontrados, segundo as normas da ABNT (Associação Brasileira de Normas Técnicas).
        Para cada um dos seguintes artigos científicos encontrados, sua tarefa é fornecer a referência bibliográfica completa, seguindo rigorosamente a NBR 6023 da ABNT.
        Certifique-se de incluir todos os elementos necessários na ordem correta: Sobrenome(s) do(s) autor(es) em letras maiúsculas, seguido(s) da(s) inicial(is) do(s) prenome(s);
        Título completo do artigo científico; Título do periódico (revista científica) por extenso ou abreviado conforme a norma; Local de publicação (cidade);
        Volume, número ou fascículo do periódico, se houver; Página inicial e final do artigo; Data de publicação; Link para acesso online (DOI - Digital Object Identifier - preferencialmente, ou URL),
        se disponível, precedido da expressão 'Disponível em:' e seguido de 'Acesso em: ', (ex: Acesso em: 17 maio. 2025).
        Preste atenção à autoria dos artigos na hora de formar a referência científica. Os elementos usados para as referências (como autor, título, etc.) são aqueles disponíveis no próprio artigo.
        """,
        description="Agente formatador de referências em ABNT."
    )
    entrada_do_agente_formatador_de_referencias = f"Referências: {artigos_buscados}\nData de hoje: {data_de_hoje}"
    referencias = call_agent(formatador, entrada_do_agente_formatador_de_referencias)
    return referencias

In [58]:
###########################################################
# --- Agente 5: Escritor de uso de citações --- #
###########################################################
def agente_escritor(topico, citacoes_destacadas):
    escritor = Agent(
        name="agente_escritor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator acadêmico experiente em integrar citações diretas de artigos científicos em textos analíticos, sem reescrever o significado original.
        Com base nas seguintes citações diretas destacadas anteriormente e formatadas em ABNT, sua tarefa é selecionar as 8 citações que você considera mais cruciais
        e com maior impacto científico para o tópico de pesquisa do usuário. Para cada uma dessas 8 citações, apresente um exemplo prático e conciso de como ela poderia
        ser utilizada em um texto acadêmico científico. Cada exemplo deve conter: 1. Apresentação da Citação Científica: Insira a citação direta formatada em ABNT, reproduzindo-a
        fielmente. 2. Análise Conectiva Científica: Imediatamente após a citação, escreva uma breve análise (de 100 palavras) que explique a relevância científica da citação para o tópico,
        conectando-a a outros conceitos científicos, argumentos científicos ou resultados científicos. Demonstre como essa citação poderia ser usada para sustentar um ponto de vista
        científico, introduzir um conceito científico fundamental ou evidenciar um resultado científico importante na pesquisa do usuário, sem alterar o sentido original da citação.
        """,
        description="Agente escritor de exemplos de uso de citações."
    )
    entrada_do_agente_escritor = f"Tópico: {topico}\nCitações: {citacoes_destacadas}"
    ex_uso_de_citacoes = call_agent(escritor, entrada_do_agente_escritor)
    return ex_uso_de_citacoes

In [64]:
############################################
# --- Agente 6: Conector temático --- #
############################################
def agente_conector_tematico(topico, exemplos_citacoes):
    conector = Agent(
        name="agente_conector_tematico",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
        Você é um especialista em identificar conexões temáticas e sugerir leituras complementares relevantes em pesquisa científica acadêmica.
        Considerando o tópico inicial do usuário e os exemplos práticos de uso de citações de artigos científicos fornecidos pelo Agente 5,
        sua tarefa é identificar e sugerir pelo menos 3 outros artigos científicos (além dos já encontrados pelo Agente 1) que possam contribuir
        de forma significativa para a compreensão e o aprofundamento científico do tema abordado. Para cada artigo científico sugerido, forneça
        as seguintes informações: 1. Título do Artigo Científico: O título completo do artigo científico. 2. Link (se disponível): O link direto
        para o artigo científico. 3. Justificativa da Relevância Científica: Explique em mais ou menos 1 parágrafo como esse artigo científico se relaciona com
        o tópico principal e com os pontos levantados nas citações do Agente 5, com foco em sua contribuição científica. Mencione se ele aborda aspectos
        científicos similares, oferece perspectivas científicas diferentes ou aprofunda algum dos conceitos científicos chave.
        """,
        description="Agente conector temático de artigos científicos."
    )
    entrada_do_agente_conector_tematico = f"Tópico: {topico}\nExemplos de uso de citações: {exemplos_citacoes}"
    conexoes_tematicas = call_agent(conector, entrada_do_agente_conector_tematico)
    return conexoes_tematicas

In [60]:
############################################
# --- Agente 7: Análisador de sentimentos --- #
############################################
def agente_analisador_de_sentimentos(topico, artigos_buscados):
    analisador = Agent(
        name="agente_analisador_de_sentimentos",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em análise de sentimento em textos científicos. Sua tarefa é analisar o sentimento geral expresso nos artigos encontrados pelo buscador.
        Ao analisar o texto, você deve identificar se o sentimento predominante é:
          - Positivo: indicando concordância, otimismo, aprovação, ou resultados favoráveis.
          - Negativo: indicando discordância, pessimismo, crítica, ou resultados desfavoráveis.
          - Neutro: indicando objetividade, ausência de emoção clara, ou apresentação de fatos sem julgamento.
        Além de identificar o sentimento geral, você deve estar atento a nuances ou diferentes sentimentos que possam ser expressos em partes específicas do texto.
        Se identificar tais nuances, mencione-as brevemente em sua resposta, indicando a que parte do texto elas se referem (se possível, de forma concisa).
        Sua resposta deve ser concisa e clara, indicando o sentimento geral. Se relevante, inclua as nuances identificadas. Lembre-se também de incluir uma análise relevante para o topico digitado pelo usuário.
        Exemplos de formato de resposta:
          - Sentimento geral: Positivo.
          - Sentimento geral: Negativo, com críticas específicas à metodologia utilizada na seção de análise.
          - Sentimento geral: Neutro, apresentando os resultados da pesquisa de forma puramente objetiva.
          - Sentimento geral: Misto. A introdução apresenta um tom otimista, enquanto a discussão dos resultados expressa algumas preocupações.
        """,
        description="Agente analisador do sentimento contido nos artigos."
    )
    entrada_do_agente_analisador_de_sentimentos = f"Tópico: {topico}\nTom e sentimentos dos artigos: {artigos_buscados}"
    tom_dos_artigos = call_agent(analisador, entrada_do_agente_analisador_de_sentimentos)
    return tom_dos_artigos

In [61]:
############################################
# --- Agente 8: Intrutor de pesquisa --- #
############################################
def agente_instrutor_de_pesquisa(topico, artigos_buscados):
    instrutor = Agent(
        name="agente_instrutor_de_pesquisa",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em pesquisa científica e sua tarefa é auxiliar um pesquisador a expandir sua investigação a partir dos artigos que já foram encontrados sobre um determinado tópico.
        Com base na análise dos títulos dos artigos que o já fornecidos pelo buscador, sua missão é identificar possíveis lacunas no conhecimento, áreas que demandam maior exploração e temas relacionados que podem enriquecer a pesquisa.
        Sugira pelo menos 5 próximos passos concretos e abrangentes para o pesquisador avançar em sua jornada investigativa. Estas sugestões devem ir além do óbvio e estimular uma exploração mais profunda e diversificada do tema.
        Considere as seguintes categorias ao formular suas sugestões:
          1. Expansão da Busca por Palavras-Chave e Conceitos: Quais termos relacionados ou sinônimos o pesquisador poderia explorar em futuras buscas? Existem conceitos adjacentes ou sub-tópicos que merecem investigação?
          Sugira combinações de palavras-chave que poderiam refinar ou ampliar o escopo da pesquisa.
          2. Diversificação das Fontes de Pesquisa e Repositórios: Além das bases de dados já utilizadas, quais outros repositórios acadêmicos, bibliotecas digitais ou plataformas de indexação poderiam ser relevantes (e confiáveis)?
          3. Exploração de Metodologias e Abordagens Alternativas: Quais outras metodologias de pesquisa (qualitativa, quantitativa, métodos mistos, estudos de caso, experimentos, etc.) poderiam ser aplicadas ao tópico?
          4. Investigação de Contextos e Variáveis Adicionais: Em quais diferentes populações, contextos geográficos, culturais ou temporais o tópico poderia ser investigado? Quais variáveis adicionais (demográficas, socioeconômicas, ambientais, etc.) poderiam influenciar ou ser influenciadas pelo fenômeno em estudo?
          5. Análise da Evolução Histórica e Tendências Futuras: Existem marcos históricos importantes na pesquisa? Existem debates ou controvérsias emergentes?
        Você também deve fornecer exemplos concretos e específicos sempre que possível para orientar o pesquisador em seus próximos passos, assim como umaa descrição do passo de forma fácil de ser entendida.
        """,
        description="Agente que ajuda a definir os próximos passos de pesquisa."
    )
    entrada_do_agente_instrutor_de_pesquisa = f"Tópico: {topico}\nPróximos passos de pesquisa: {artigos_buscados}"
    proximos_passos = call_agent(instrutor, entrada_do_agente_instrutor_de_pesquisa)
    return proximos_passos

In [62]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o LUMINA:seu Sistema de Busca e Análise de Artigos Científicos 🚀")
print("Olá! Eu sou a LUMINA, sua assistente favorita na hora de iluminar a sua busca por artigos científicos de forma rápida 😁😊!")
print("----------------------------------------------------------------------------------------------------------")

# --- Obtendo o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer buscar artigos científicos: ")

# Logística dos agentes ################################################
while topico != "Tchau Lumina":
    if not topico:
        print("Você esqueceu de digitar o seu tópico!")
        topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer buscar artigos científicos: ")
    else:
        print(f"Maravilha! Vamos então buscar e analisar artigos sobre {topico}")

        artigos_buscados = agente_buscador(topico, data_de_hoje)
        print("\n--- 📝 Resultados da busca de artigos ---\n")
        display(Markdown(artigos_buscados))
        print("--------------------------------------------------------------")

        resumos = agente_redator_de_resumos(topico, artigos_buscados)
        print("\n--- 📝 Resumo dos artigos encontrados ---\n")
        display(Markdown(resumos))
        print("--------------------------------------------------------------")

        sentimentos = agente_analisador_de_sentimentos(topico, artigos_buscados)
        print("\n--- 📝 Sentimento dos artigos encontrados ---\n")
        display(Markdown(sentimentos))
        print("--------------------------------------------------------------")

        citacoes = agente_captador_de_citacoes(topico, artigos_buscados)
        print("\n--- 📝 Citações relevantes dos artigos ---\n")
        display(Markdown(citacoes))
        print("--------------------------------------------------------------")

        referencias_abnt = agente_formatador_de_referencias(artigos_buscados, data_de_hoje)
        print("\n--- 📝 Referências em ABNT ---\n")
        display(Markdown(referencias_abnt))
        print("--------------------------------------------------------------")

        exemplos_de_uso = agente_escritor(topico, citacoes)
        print("\n--- 📝 Exemplo de uso das citações) ---\n")
        display(Markdown(exemplos_de_uso))
        print("--------------------------------------------------------------")

        conexoes_tematicas = agente_conector_tematico(topico, exemplos_de_uso)
        print("\n--- 📝 Outros artigos relacionados à temática ---\n")
        display(Markdown(conexoes_tematicas))
        print("--------------------------------------------------------------")

        proximos_passos = agente_instrutor_de_pesquisa(topico, artigos_buscados)
        print("\n--- 📝 Próximos passos de pesquisa ---\n")
        display(Markdown(proximos_passos))

        print("\n✨ Processo de análise de artigos concluído! ✨")
        topico = input("❓ Digite outro tópico para pesquisar (ou 'Tchau Lumina' para sair): ")

print("\n👋 Tchau! Espero ter ajudado a iLUMINAr suas pesquisas!")

🚀 Iniciando o LUMINA:seu Sistema de Busca e Análise de Artigos Científicos 🚀
Olá! Eu sou a LUMINA, sua assistente favorita na hora de iluminar a sua busca por artigos científicos de forma rápida 😁😊!
----------------------------------------------------------------------------------------------------------
❓ Por favor, digite o TÓPICO sobre o qual você quer buscar artigos científicos: IA
Maravilha! Vamos então buscar e analisar artigos sobre IA

--- 📝 Resultados da busca de artigos ---



Certo, vou realizar uma busca abrangente por artigos científicos sobre Inteligência Artificial (IA), priorizando publicações dos últimos 10 anos e fontes confiáveis como SciELO, LILACS, PubMed, Web of Science e Scopus.


Com certeza! Aqui estão 8 artigos científicos sobre Inteligência Artificial (IA), publicados nos últimos 10 anos, priorizando fontes confiáveis:

1.  **Título:** Inteligência artificial: passado, aplicações atuais e futuro.
    *   **Autores:** Inatel
    *   **Data de Publicação:** 16 de setembro de 2024
    *   **Periódico:** Notícias - Inatel
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEV8xoTUvdkJDH1Qk1\_QMs0kkkbtW7kUCbmRUlnHpfhzI85OMZJzf1l\_eM9dJsV60fOiiq7JMvZz2ywmJJ2z8arbWKXQmxT1x3rx97-FRiw9IFcJRaqCbOKz12bQ\_yL\_OQaBS\_zalSrjDj\_wcF7IPxqNKPKUtxyxf8VS6Zy3oi1xicyD9YMksDl0qNB-SebMAc=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEV8xoTUvdkJDH1Qk1_QMs0kkkbtW7kUCbmRUlnHpfhzI85OMZJzf1l_eM9dJsV60fOiiq7JMvZz2ywmJJ2z8arbWKXQmxT1x3rx97-FRiw9IFcJRaqCbOKz12bQ_yL_OQaBS_zalSrjDj_wcF7IPxqNKPKUtxyxf8VS6Zy3oi1xicyD9YMksDl0qNB-SebMAc=)

2.  **Título:** PUBLICAÇÃO DE ARTIGOS CIENTÍFICOS E IA: UMA REVOLUÇÃO EM CURSO.
    *   **Autores:** SciELO
    *   **Data de Publicação:** 13 de maio de 2024
    *   **Periódico:** SciELO.org - Rede SciELO
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHaUDo3ReYpAxuejeojUqIE7zs1c2jp974fxx93fm9iEuzRaMT0ghgY-4kvAnInIDmXVyBfTCHwkBUJtPnwf3uP50\_SsNaRf2V-5MIRrPT7WB4cvZdxsm94tB1lmV6N3ULfi1oTYtDxe\_4ZR3szh2ona2Pc9pBZpfI=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHaUDo3ReYpAxuejeojUqIE7zs1c2jp974fxx93fm9iEuzRaMT0ghgY-4kvAnInIDmXVyBfTCHwkBUJtPnwf3uP50_SsNaRf2V-5MIRrPT7WB4cvZdxsm94tB1lmV6N3ULfi1oTYtDxe_4ZR3szh2ona2Pc9pBZpfI=)

3.  **Título:** A evolução da inteligência artificial e suas aplicações para investidores institucionais
    *   **Autor:** Rodrigo Maranhão
    *   **Data de Publicação:** 7 de julho de 2023
    *   **Periódico:** Abrapp em Foco
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXES0WfIUmfqPfQlIGothDQFAV7Rj-tJA5Rtz\_a1UpczpI-a1rMhhxTEpTdhUNfnpWFl5gq12zIAM921HQsU0g7fcoJwcbLT7Cbzdr04c6TMEu-flR9ua44Sx6X56t\_8uc6nzq1HHHb6JGEUU5w08deT5QyeH93fX9CnYw6uZEim8xXarQgwVMSqBsKZzqRqEw6WiWE4wiMW7ahNjHOQeX9yj190lmvzBAudJ9HZOHxrRU1c7lCYDFHG4TPwaMNxgq5esWQv5EyDqKDTeyHl7TrlCQ==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXES0WfIUmfqPfQlIGothDQFAV7Rj-tJA5Rtz_a1UpczpI-a1rMhhxTEpTdhUNfnpWFl5gq12zIAM921HQsU0g7fcoJwcbLT7Cbzdr04c6TMEu-flR9ua44Sx6X56t_8uc6nzq1HHHb6JGEUU5w08deT5QyeH93fX9CnYw6uZEim8xXarQgwVMSqBsKZzqRqEw6WiWE4wiMW7ahNjHOQeX9yj190lmvzBAudJ9HZOHxrRU1c7lCYDFHG4TPwaMNxgq5esWQv5EyDqKDTeyHl7TrlCQ==)

4.  **Título:** INTELIGÊNCIA ARTIFICIAL NA EDUCAÇÃO: CONTRIBUIÇÕES E DESAFIOS PARA O PROCESSO DE ENSINO E APRENDIZAGEM.
    *   **Autores:** Adriana Queli de Freitas, Lílian Andrade do Rêgo, Madeanne Sousa Costa, Cassandra Paula Sales Linhares Monteiro, Waldenisia Dias da Pascoa Costa
    *   **Data de Publicação:** 12 de fevereiro de 2025
    *   **Periódico:** REVISTA FOCO
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG9TuuvW3ZM8FLR1XOgQbBVhDNLFlXoathWL3PWKl11mNR4fPmcWYSCHyYPAbau8Mk971BF\_8u2COEAB-3JTg4sUeppTO7UD3ChqbxnS7v562LRfgsg2HdgFu1MrYB9jMdldeWGmI4dcuoqoSTPQnPO4zNLPw==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG9TuuvW3ZM8FLR1XOgQbBVhDNLFlXoathWL3PWKl11mNR4fPmcWYSCHyYPAbau8Mk971BF_8u2COEAB-3JTg4sUeppTO7UD3ChqbxnS7v562LRfgsg2HdgFu1MrYB9jMdldeWGmI4dcuoqoSTPQnPO4zNLPw==)

5.  **Título:** IMPACTO DA IA NA PRODUTIVIDADE CIENTÍFICA: UMA REVISÃO SISTEMATIZADA.
    *   **Autores:** REVISTA FOCO
    *   **Data de Publicação:** 7 de janeiro de 2025
    *   **Periódico:** REVISTA FOCO
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE1N09wY8KOfah2AeAPU3C6CNgPe0CUowd4qw9GDGNC9lrTL5A9TY\_xowMCCxZRH9Uj3u3ryY4R\_3w0eYtAny-sWeO4gaTH01veb2nBFodjFbzrENQsNm60uR2HRF96EaCFInTP74RDapoK2w3-rYrj6DJnEA==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE1N09wY8KOfah2AeAPU3C6CNgPe0CUowd4qw9GDGNC9lrTL5A9TY_xowMCCxZRH9Uj3u3ryY4R_3w0eYtAny-sWeO4gaTH01veb2nBFodjFbzrENQsNm60uR2HRF96EaCFInTP74RDapoK2w3-rYrj6DJnEA==)

6.  **Título:** Inteligência artificial como ferramenta de auxílio ao diagnóstico de câncer de pele.
    *   **Autores:** Amanda Maria Cavalcante Soares, Milena Nunes Alves de Sousa
    *   **Data de Publicação:** 6 de janeiro de 2025
    *   **Periódico:** Revista Brasileira de Educação e Saúde
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGn-q3JV1jK5Fc4uy1gUDNRTXDxJnDMpCBFhx5wkQOpgH5K84sNulrcBWhuA0-Ku3k5\_ySIF-hSJzfCf2dgvG6ZKOMVWEtb2dLobB2qr8ONRYbL\_nLw3fzqhgzKj9g2M7pzBXCq9AKkahlonvUhzFFKiQmertp0YtSn8FY3-w==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGn-q3JV1jK5Fc4uy1gUDNRTXDxJnDMpCBFhx5wkQOpgH5K84sNulrcBWhuA0-Ku3k5_ySIF-hSJzfCf2dgvG6ZKOMVWEtb2dLobB2qr8ONRYbL_nLw3fzqhgzKj9g2M7pzBXCq9AKkahlonvUhzFFKiQmertp0YtSn8FY3-w==)

7.  **Título:** Avanços na assistência de enfermagem: uma revisão de escopo sobre o uso da Inteligência Artificial na ciência do cuidar.
    *   **Autores:** Débora Regina Alves Raposo, Josivan Soares Alves Júnior, Débora Taynah Oliveira da Silva, Rayane de Almeida Alves, Danielle Christine Moura dos Santos, Thayse Mota Alves, Joyce Kell Sampaio da Silva, Samara Farias Aragão, Cosme Michael Santos Farias
    *   **Data de Publicação:** 26 de julho de 2024
    *   **Periódico:** CONTRIBUCIONES A LAS CIENCIAS SOCIALES
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHKYkdDsFVx0ETx1xeEF9pUrOcMxqWfGSrZKquad1h33DULO1AIMfOHjxozKib5eFfaOKb1Y-tF\_PNPLMrBZwuk0xfmgFn7cUEDXrmqEjpVc8Sc63frYoWvth\_8mGhAGW\_Idmq2IuBBC761XXwv\_o5Ng-ifZKalSTk7A41LDjKUC5VFjyi3](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHKYkdDsFVx0ETx1xeEF9pUrOcMxqWfGSrZKquad1h33DULO1AIMfOHjxozKib5eFfaOKb1Y-tF_PNPLMrBZwuk0xfmgFn7cUEDXrmqEjpVc8Sc63frYoWvth_8mGhAGW_Idmq2IuBBC761XXwv_o5Ng-ifZKalSTk7A41LDjKUC5VFjyi3)

8.  **Título:** Inteligência artificial no contexto da Ciência da Informação.
    *   **Autores:** Fernanda Maria Melo Alves, Maria do Carmo de Oliveira Ribeiro.
    *   **Data de Publicação:** 26 de abril de 2024
    *   **Periódico:** Portal de Revistas da ABECIN
    *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGH1criTd9toAv2ZGNb152u2H9wB4VINMfsNxzwp4ydTeFGouf7TCmC5vDXg37601-oYnj61gZ2HG1wXEWNsip1piqxjyy0mVk2IHhsnN54R7Aip5OHSb6NuourYuotunolYTRIqFvIKonxXsGCNF3BhUY=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGH1criTd9toAv2ZGNb152u2H9wB4VINMfsNxzwp4ydTeFGouf7TCmC5vDXg37601-oYnj61gZ2HG1wXEWNsip1piqxjyy0mVk2IHhsnN54R7Aip5OHSb6NuourYuotunolYTRIqFvIKonxXsGCNF3BhUY=)

Espero que esta seleção seja útil para sua pesquisa!


--------------------------------------------------------------

--- 📝 Resumo dos artigos encontrados ---



Aqui estão os resumos dos artigos científicos sobre Inteligência Artificial (IA) fornecidos:

1.  **Título:** Inteligência artificial: passado, aplicações atuais e futuro.
    Este artigo explora a trajetória da inteligência artificial, desde suas origens até as aplicações contemporâneas e as perspectivas futuras. O objetivo é fornecer uma visão geral do campo da IA, abordando seu desenvolvimento histórico, as tecnologias e usos atuais em diversas áreas, e as tendências que moldarão seu futuro. O artigo destaca como a IA está evoluindo e se integrando em diferentes aspectos da sociedade, impactando a tecnologia e a inovação.

2.  **Título:** PUBLICAÇÃO DE ARTIGOS CIENTÍFICOS E IA: UMA REVOLUÇÃO EM CURSO.
    Este artigo discute o impacto da inteligência artificial (IA) no processo de publicação de artigos científicos. O objetivo é analisar como a IA está transformando a forma como a pesquisa científica é conduzida e divulgada. O artigo aborda o uso de ferramentas de IA na revisão, edição e disseminação de artigos, bem como as implicações éticas e práticas dessa revolução tecnológica. A discussão visa fornecer uma visão geral das oportunidades e desafios que a IA traz para a comunidade científica.

3.  **Título:** A evolução da inteligência artificial e suas aplicações para investidores institucionais
    O artigo explora a evolução da inteligência artificial (IA) e suas aplicações no contexto de investimentos institucionais. O objetivo é analisar como a IA está sendo utilizada por investidores para melhorar a tomada de decisões e otimizar estratégias de investimento. O artigo destaca as diferentes áreas em que a IA pode ser aplicada, como análise de dados, previsão de mercado e gestão de portfólio, e discute os benefícios e desafios dessa tecnologia para o setor financeiro.

4.  **Título:** INTELIGÊNCIA ARTIFICIAL NA EDUCAÇÃO: CONTRIBUIÇÕES E DESAFIOS PARA O PROCESSO DE ENSINO E APRENDIZAGEM.
    Este artigo examina as contribuições e desafios da inteligência artificial (IA) no processo de ensino e aprendizagem. O objetivo é analisar como a IA está sendo utilizada na educação, abordando suas aplicações no desenvolvimento de ferramentas pedagógicas, personalização do ensino e avaliação do desempenho dos alunos. O artigo também discute os desafios éticos e práticos associados ao uso da IA na educação, como a necessidade de garantir a equidade, a privacidade e a segurança dos dados dos alunos.

5.  **Título:** IMPACTO DA IA NA PRODUTIVIDADE CIENTÍFICA: UMA REVISÃO SISTEMATIZADA.
    O artigo apresenta uma revisão sistematizada sobre o impacto da inteligência artificial (IA) na produtividade científica. O objetivo é analisar como a IA está afetando a forma como a pesquisa científica é conduzida, desde a coleta e análise de dados até a redação e publicação de artigos. O estudo busca identificar os benefícios e desafios do uso da IA na pesquisa, bem como as áreas em que essa tecnologia tem o maior potencial de impacto.

6.  **Título:** Inteligência artificial como ferramenta de auxílio ao diagnóstico de câncer de pele.
    Este artigo aborda o uso da inteligência artificial (IA) como ferramenta de auxílio ao diagnóstico de câncer de pele. O objetivo é investigar como a IA pode ser utilizada para melhorar a precisão e a eficiência do diagnóstico precoce do câncer de pele, através da análise de imagens e dados clínicos. O estudo destaca os benefícios potenciais da IA para a detecção do câncer de pele, bem como os desafios e limitações dessa tecnologia.

7.  **Título:** Avanços na assistência de enfermagem: uma revisão de escopo sobre o uso da Inteligência Artificial na ciência do cuidar.
    Este artigo apresenta uma revisão de escopo sobre o uso da inteligência artificial (IA) na assistência de enfermagem. O objetivo é examinar como a IA está sendo aplicada na ciência do cuidar, abordando suas aplicações no monitoramento de pacientes, na tomada de decisões clínicas e na otimização dos processos de trabalho da enfermagem. A revisão destaca os avanços e desafios do uso da IA na enfermagem, bem como suas implicações para a qualidade e a segurança do cuidado ao paciente.

8.  **Título:** Inteligência artificial no contexto da Ciência da Informação.
    Este artigo explora o papel da inteligência artificial (IA) no contexto da Ciência da Informação. O objetivo é analisar como a IA está sendo utilizada para melhorar a organização, o acesso e a disseminação da informação, bem como suas implicações para a pesquisa e a prática da Ciência da Informação. O artigo aborda o uso da IA em diferentes áreas, como a recuperação da informação, a análise de dados e a gestão do conhecimento, e discute os desafios e oportunidades dessa tecnologia para a área.



--------------------------------------------------------------

--- 📝 Sentimento dos artigos encontrados ---



...

--------------------------------------------------------------

--- 📝 Outros artigos relacionados à temática ---



Com base no tópico de Inteligência Artificial (IA) e nos exemplos de uso de citações fornecidos, sugiro os seguintes artigos científicos para complementar a pesquisa:

**1. Título do Artigo Científico:** "Explainable AI (XAI): Concepts, Taxonomies, Opportunities and Challenges toward Responsible AI"
   *   **Link (se disponível):** [https://www.mdpi.com/2076-3417/12/2/488](https://www.mdpi.com/2076-3417/12/2/488)
   *   **Justificativa da Relevância Científica:** Este artigo aborda um aspecto crucial da IA, a interpretabilidade e explicabilidade. Dado que os exemplos de citações mencionam aplicações da IA em áreas sensíveis como saúde e educação, a necessidade de compreender como os algoritmos de IA chegam a suas conclusões torna-se fundamental. O artigo explora conceitos, taxonomias, oportunidades e desafios da IA Explicável (XAI), contribuindo para um desenvolvimento e uso mais responsável e ético da IA, o que complementa as discussões sobre personalização do ensino e detecção de câncer de pele, onde a transparência algorítmica é essencial.

**2. Título do Artigo Científico:** "Artificial Intelligence in Healthcare: Past, Present and Future"
   *   **Link (se disponível):** [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7588650/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7588650/)
   *   **Justificativa da Relevância Científica:** Este artigo oferece uma visão geral abrangente da aplicação da IA na área da saúde, desde o passado até as perspectivas futuras. Ele se conecta diretamente com as citações que mencionam a IA na detecção de câncer de pele e no monitoramento de pacientes. O artigo aprofunda a discussão sobre o potencial da IA para transformar o setor de saúde, abordando tanto os benefícios quanto os desafios éticos e práticos associados à sua implementação. Ao fornecer um contexto histórico e uma análise das tendências atuais, este artigo pode enriquecer a compreensão do impacto da IA na saúde.

**3. Título do Artigo Científico:** "A review of artificial intelligence in education"
   *   **Link (se disponível):** [https://www.tandfonline.com/doi/full/10.1080/09523987.2021.1916004](https://www.tandfonline.com/doi/full/10.1080/09523987.2021.1916004)
   *   **Justificativa da Relevância Científica:** Considerando as citações sobre personalização do ensino e suporte ao aluno via chatbots, este artigo de revisão oferece uma análise aprofundada das aplicações da IA na educação. Ele examina o impacto da IA em diferentes aspectos do ensino e da aprendizagem, incluindo a personalização, a avaliação, o feedback e o suporte ao aluno. Ao apresentar uma visão geral das pesquisas existentes e identificar tendências emergentes, este artigo pode fornecer insights valiosos para pesquisadores e educadores interessados em explorar o potencial da IA para transformar a educação. Além disso, discute as limitações e desafios da IA na educação.


--------------------------------------------------------------

--- 📝 Próximos passos de pesquisa ---



Com base nos títulos dos artigos fornecidos, podemos identificar alguns temas recorrentes e propor os seguintes passos para expandir sua pesquisa sobre Inteligência Artificial:

1. **Expansão da Busca por Palavras-Chave e Conceitos:**

*   **Descrição:** Explore termos relacionados e sinônimos para IA, além de conceitos adjacentes. Isso ajudará a descobrir novas áreas de pesquisa e refinar seu foco.
*   **Sugestões:**
    *   Utilize termos como "Aprendizado de Máquina", "Redes Neurais", "Processamento de Linguagem Natural", "Visão Computacional", "Robótica", "Automação Inteligente".
    *   Experimente combinações como "IA e Saúde", "IA e Educação", "IA e Finanças", "IA e Sustentabilidade", "Ética em IA", "Governança de IA", "Impacto Social da IA".
    *   Explore sub-tópicos como "IA Explicável (XAI)", "Aprendizado por Reforço", "IA Generativa", "Transfer Learning".

2. **Diversificação das Fontes de Pesquisa e Repositórios:**

*   **Descrição:** Além das bases de dados já utilizadas, explore outros repositórios acadêmicos e plataformas de indexação relevantes para garantir uma visão abrangente do tema.
*   **Sugestões:**
    *   **IEEE Xplore:** Focado em engenharia, ciência da computação e áreas relacionadas.
    *   **ACM Digital Library:** Essencial para artigos de computação e IA.
    *   **arXiv:** Repositório de preprints (artigos ainda não revisados por pares) de física, matemática, ciência da computação e áreas relacionadas.
    *   **Google Scholar:** Permite buscar em diversas fontes, mas valide a qualidade das publicações encontradas.
    *   **Repositórios de Universidades:** Muitas universidades mantêm repositórios digitais com teses, dissertações e artigos de seus pesquisadores.

3. **Exploração de Metodologias e Abordagens Alternativas:**

*   **Descrição:** Considere diferentes metodologias de pesquisa para abordar o tema da IA sob perspectivas variadas.
*   **Sugestões:**
    *   **Estudos de Caso:** Analise a aplicação da IA em setores específicos (saúde, educação, indústria) para entender seus impactos e desafios.
    *   **Revisões Sistemáticas e Meta-Análises:** Sintetize os resultados de múltiplos estudos para identificar tendências e lacunas na pesquisa.
    *   **Pesquisa Qualitativa:** Realize entrevistas com especialistas, usuários e desenvolvedores de IA para compreender suas percepções e experiências.
    *   **Experimentos:** Desenvolva experimentos para testar o desempenho de algoritmos de IA em diferentes tarefas e contextos.
    *   **Análise de Dados:** Utilize técnicas de mineração de dados e análise de redes para identificar padrões e relações em grandes conjuntos de dados relacionados à IA.

4. **Investigação de Contextos e Variáveis Adicionais:**

*   **Descrição:** Explore como a IA se manifesta e é influenciada por diferentes contextos e variáveis, aprofundando sua compreensão do tema.
*   **Sugestões:**
    *   **Contextos Geográficos:** Compare a adoção e o desenvolvimento da IA em diferentes países e regiões, considerando fatores como infraestrutura, políticas públicas e cultura.
    *   **Populações Específicas:** Investigue como a IA afeta diferentes grupos demográficos (gênero, idade, nível socioeconômico, etnia), considerando questões de equidade e inclusão.
    *   **Variáveis Socioeconômicas:** Analise a relação entre IA e indicadores como emprego, renda, desigualdade social e desenvolvimento econômico.
    *   **Variáveis Ambientais:** Explore o uso da IA para resolver problemas ambientais, como mudanças climáticas, desmatamento e poluição.

5. **Análise da Evolução Histórica e Tendências Futuras:**

*   **Descrição:** Compreenda como a pesquisa em IA evoluiu ao longo do tempo e quais são as tendências emergentes que podem moldar o futuro do campo.
*   **Sugestões:**
    *   **Marcos Históricos:** Identifique os principais avanços e desafios na história da IA, desde seus primórdios até os dias atuais.
    *   **Debates e Controvérsias:** Analise as discussões sobre questões éticas, sociais e filosóficas relacionadas à IA, como viés algorítmico, privacidade, segurança e autonomia.
    *   **Tendências Emergentes:** Acompanhe os desenvolvimentos em áreas como IA Generativa, IA Explicável (XAI), IA Auto Supervisionada e IA Quântica.
    *   **Previsões e Cenários Futuros:** Explore estudos prospectivos e relatórios de especialistas que projetam o impacto da IA em diferentes setores e na sociedade como um todo.

Lembre-se de que a pesquisa científica é um processo iterativo e dinâmico. Esteja aberto a ajustar seus objetivos e abordagens à medida que você aprende mais sobre o tema.



✨ Processo de análise de artigos concluído! ✨
❓ Digite outro tópico para pesquisar (ou 'Tchau Lumina' para sair): Tchau Lumina

👋 Tchau! Espero ter ajudado a iLUMINAr suas pesquisas!
